Pip:


In [ ]:
!pip3 install langchain streamlit watchdog pypdf unstructured pinecone-client openai tiktoken

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

Load the document

In [ ]:
loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
data = loader.load()

Split into smaller documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print(f'Now you have {len(texts)} documents')

Create Embeddings

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
# Environment will be in the index page in pinecone accounts under environment
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

OpenAI Embeddings with vector db's are 1536 https://betterprogramming.pub/openais-embedding-model-with-vector-database-b69014f04433

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

index_name = 'ai-doc-qa'

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content[:450])

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

query = "what is the collect stage of data maturity?"
docs = docsearch.similarity_search(query)

chain.run(input_documents=docs, question=query)